In [195]:
domains = dict(
    RBD=(319, 541),
    FP=(788, 806),
    HR1=(912, 984),
    HR2=(1163, 1213),
)

In [196]:
df_sars2_S = pd.read_csv("linear-bcell-epitopes-SARS2-S.csv")

In [197]:
glycosites = set([])
with open("../glycosylation/glycosites-Watanabe.txt") as f:
    for l in f:
        l = l.strip()
        if l.startswith("#"):
            continue
        if not l:
            continue
        glycosites.add(int(l))
print(glycosites)
print("%d glycosites" % len(glycosites))


{1158, 17, 657, 149, 1173, 282, 801, 165, 1194, 1074, 61, 709, 74, 331, 1098, 717, 343, 603, 616, 234, 1134, 122}
22 glycosites


In [198]:
df_accessibility = pd.read_csv("../solvent-accessibility/Woods-Glycans-MD-Site-Specific-Accessibility.csv")
df_accessibility

,Residue Proper Numbering,SiteSpecific Accessiblity,Nude Accessibility,Difference
0,27,0.94,0.99,0.05
1,28,0.99,1.00,0.01
2,29,0.15,0.26,0.11
3,30,0.36,0.76,0.40
4,31,0.00,0.00,0.00
...,...,...,...,...
1115,1142,0.78,0.89,0.11
1116,1143,0.66,0.70,0.04
1117,1144,0.78,0.79,0.01
1118,1145,0.85,0.86,0.00


In [199]:
accessible_positions = set([])
aa_num_to_nude_accessibility = {
    aa_num: accessibility
    for (aa_num, accessibility) 
    in zip(
        df_accessibility["Residue Proper Numbering"],
        df_accessibility["Nude Accessibility"])
}

aa_num_to_glycosylated_accessibility = {
    aa_num: accessibility
    for (aa_num, accessibility) 
    in zip(
        df_accessibility["Residue Proper Numbering"],
        df_accessibility["SiteSpecific Accessiblity"])
}


In [200]:
accessibility_threshold = 0.25
min_accessility_kmer = 3
start = 1
end = 1273

# determine accessible kmers
n_accessible = 0
accessible_positions = set([])
for aa_num in range(start, end + 1):
    accessibility = aa_num_to_glycosylated_accessibility.get(aa_num, 0.0)
    if (accessibility > accessibility_threshold) or aa_num == end:
        n_accessible += 1
    else:
        if n_accessible >= min_accessility_kmer:
            for accessible_aa_num in range(aa_num - n_accessible, aa_num):
                accessible_positions.add(accessible_aa_num)
        n_accessible = 0
print("%d accessible positions" % len(accessible_positions))


210 accessible positions


In [201]:

df_polymorphic = pd.read_csv("../polymorphism/spike_protein_entropy_above_1_per_thousand.csv")
polymorphic_sites = set(df_polymorphic.aa_num)
assert 614 in polymorphic_sites
print("%d polymorphic sites" % len(polymorphic_sites))

27 polymorphic sites


In [202]:
longest_accessible_subsequences = []
accessible_starts = []
accessible_ends = []

min_accessibility_end_of_epitope = 0.3
min_accessibility_in_epitope = 0.15
min_mean_accessiblity = 0.35
min_max_accessibility = 0.5
for (epitope_start, seq) in zip(df_sars2_S.Start, df_sars2_S.Sequence):
    epitope_end = epitope_start + len(seq) - 1
    
    longest_subsequence = ""
    longest_subsequence_length = 0
    longest_start = 0
    longest_end = 0
    for start_pos in range(epitope_start, epitope_end + 1):
        for end_pos in range(start_pos, epitope_end + 1):
            length = end_pos - start_pos + 1
            if length <= longest_subsequence_length:
                continue
            if ((aa_num_to_glycosylated_accessibility.get(start_pos, 0) >= min_accessibility_end_of_epitope) and (
                    aa_num_to_glycosylated_accessibility.get(end_pos, 0) >= min_accessibility_end_of_epitope)):
                accessibility_scores = [
                    aa_num_to_glycosylated_accessibility.get(pos, 0)
                    for pos in range(start_pos, end_pos + 1)
                ]
                mean_accessiblity = np.mean(accessibility_scores)
                min_accessibility = np.min(accessibility_scores)
                max_accessibility = np.max(accessibility_scores)
                if ((mean_accessiblity >= min_mean_accessiblity) and 
                        (min_accessibility >= min_accessibility_in_epitope) and 
                        (max_accessibility >= min_max_accessibility)):
                    longest_start = start_pos
                    longest_end = end_pos
                    longest_subsequence_length = length
                    i = start_pos - epitope_start
                    j = end_pos - epitope_start
                    longest_subsequence = seq[i:j + 1]
                    assert len(longest_subsequence) == longest_subsequence_length, (
                        longest_subsequence,
                        longest_subsequence_length)
    longest_accessible_subsequences.append(longest_subsequence)
    accessible_starts.append(longest_start)
    accessible_ends.append(longest_end)
    
df_sars2_S["accessible_subsequence"] = longest_accessible_subsequences
df_sars2_S["accessible_subsequence_start"] = accessible_starts
df_sars2_S["accessible_subsequence_end"] = accessible_ends
df_sars2_S["accessible_subsequence_length"] = df_sars2_S["accessible_subsequence"].str.len()
df_sars2_S["accessible_subsequence_is_4mer_or_longer"] = df_sars2_S["accessible_subsequence_length"] >= 4


In [203]:

df_sars2_S["accessible_subsequence_contains_glycosite"] = [
    any([i in glycosites for i in range(start, end + 1)])
    for (start, end) in zip(df_sars2_S["accessible_subsequence_start"], df_sars2_S["accessible_subsequence_end"])
];

df_sars2_S["accessible_subsequence_contains_polymorphism"] = [
    any([i in polymorphic_sites for i in range(start, end + 1)])
    for (start, end) in zip(df_sars2_S["accessible_subsequence_start"], df_sars2_S["accessible_subsequence_end"])
];

def dist(epitope_start, epitope_end, feature_start, feature_end):
    """
    Distance between two intervals, defined as 0 if any 
    epitope AAs contained in feature
    """
    if epitope_end <= feature_start:
        return feature_start - epitope_end
    elif epitope_start >= feature_end:
        return epitope_start - feature_end 
    else:
        return 0
        
exact_overlap_columns = []
padded_overlap_columns = []
for feature, (feature_start, feature_end) in domains.items():
    padding = 30 if feature == "RBD" else 15
    exact_distance_column_name = "distance_to_%s" % feature
    df_sars2_S[exact_distance_column_name] = [
        dist(start, end, feature_start, feature_end) 
        for (start, end)
        in zip(df_sars2_S.Start, df_sars2_S.End)
    ]
    exact_overlap_column_name = "in_%s" % feature
    df_sars2_S["in_%s" % feature] = df_sars2_S[exact_distance_column_name] == 0
    exact_overlap_columns.append(exact_overlap_column_name)
    padded_distance_column_name = exact_distance_column_name + "_padded"
    df_sars2_S[padded_distance_column_name] = [
        dist(start, end, feature_start - padding, feature_end + padding) 
        for (start, end)
        in zip(df_sars2_S.Start, df_sars2_S.End)
    ]
    padded_overlap_column_name = "near_%s" % feature
    df_sars2_S[padded_overlap_column_name] = df_sars2_S[padded_distance_column_name] == 0
    padded_overlap_columns.append(padded_overlap_column_name)
        
in_any_feature = np.zeros(len(df_sars2_S), dtype=bool)
for col in exact_overlap_columns:
    in_any_feature |= df_sars2_S[col]
df_sars2_S["in_any_feature"] = in_any_feature

near_any_feature = np.zeros(len(df_sars2_S), dtype=bool)
for col in padded_overlap_columns:
    near_any_feature |= df_sars2_S[col]
df_sars2_S["near_any_feature"] = near_any_feature

df_sars2_S["IgA"] = df_sars2_S.Isotype.str.contains("IgA")
df_sars2_S["IgG"] = df_sars2_S.Isotype.str.contains("IgG")
df_sars2_S["IgG_and_IgA"] = df_sars2_S["IgG"] & df_sars2_S["IgA"];


In [204]:
df_sars2_S["accessible_subsequence"]

0                    TE
1                  PSKP
2                  AYTN
3                    FK
4                      
5                  RKSN
6                  PSKP
7                      
8                      
9                      
10      SNLKPFERDISTEIY
11                   YQ
12                   YQ
13                 QTLE
14                 QTLE
15                 QTLE
16                     
17                 PSKP
18                 PSKP
19                     
20                     
21                   QF
22                   QF
23                   TR
24         VAIHADQLTPTW
25             EHVNNSYE
26                     
27                  GTH
28                QPELD
29                     
30                     
31        IHVSGTNGTKRFD
32       YYHKNNKSWMESEF
33                   QG
34    HRSYLTPGDSSSGWTAG
35                    E
36                   TR
37                     
38              RKSNLKP
39                 QTLE
40                   TE
41             E

In [205]:
df_sars2_S.to_csv("linear-bcell-epitopes-SARS2-S-with-filters.csv")

In [206]:
from collections import OrderedDict

inherited_col_names = [
    "accessible_subsequence_length",
    "accessible_subsequence_is_4mer_or_longer",
    "IgG",
    "IgA",
    "IgG_and_IgA",
    "accessible_subsequence_contains_polymorphism",
    "accessible_subsequence_contains_glycosite",
    "near_any_feature",
    "in_any_feature",
     
]
key_col_names = [

    "accessible_subsequence_start",
    "accessible_subsequence_end", 
    "accessible_subsequence", 
]
combined_col_names = key_col_names + inherited_col_names
grouped_cols = OrderedDict(
    [("num_sources", []), ("sources", [])] + [(col_name, []) for col_name in combined_col_names]
)

for key, group in \
        sorted(df_sars2_S.groupby([
            "accessible_subsequence_start", 
            "accessible_subsequence_end",
            "accessible_subsequence"])):
    if key[0] == 0:
        # skip epitopes without any accessible parts
        continue
    for (key_elt, key_col_name) in zip(key, key_col_names):
        grouped_cols[key_col_name].append(key_elt)
    for col_name in inherited_col_names:
        value = group[col_name].mean()
        if "length" in col_name:
            value = int(value)
        elif "Ig" not in col_name:
        
            value = bool(value)
        grouped_cols[col_name].append(value)
    grouped_cols["num_sources"].append(len(group))
    grouped_cols["sources"].append("; ".join([
        "%s %s%d-%d" % (source, protein, start, end) for
        (source, protein, start, end) in 
        zip(group.Source, group.Protein, group.Start, group.End)
    ]))
df_grouped = pd.DataFrame(grouped_cols);

df_grouped.to_csv("accessible-linear-bcell-epitopes-grouped-by-sequence.csv", index=False)
print("%d entries grouped by accessible sequence" % (
    len(df_grouped)))

26 entries grouped by accessible sequence


In [207]:
def combine_overlapping_epitopes(df_grouped):
    merged_intervals = []
    candidate_intervals = [
        (start, end, seq, {i}) 
        for i, (start, end, seq) in 
            enumerate(
                zip(df_grouped.accessible_subsequence_start, 
                    df_grouped.accessible_subsequence_end,
                    df_grouped.accessible_subsequence))
    ]
    candidate_intervals = sorted(candidate_intervals, key=lambda x: (x[0], x[1]))
    
    merged_start, merged_end, merged_sequence, merged_indices = candidate_intervals[0]
    for (curr_start, curr_end, curr_seq, curr_indices) in candidate_intervals[1:]:
        if curr_start <= merged_end:
            merged_indices.update(curr_indices)
            n_extra_chars = curr_end - merged_end
            if n_extra_chars > 0:
                merged_sequence += curr_seq[-n_extra_chars:]
            merged_end = max(merged_end, curr_end)
            
        else:
            merged_intervals.append((merged_start, merged_end, merged_sequence, merged_indices))
            merged_start = curr_start
            merged_end = curr_end
            merged_indices = curr_indices
            merged_sequence = curr_seq
    merged_intervals.append((merged_start, merged_end, merged_sequence, merged_indices))
    result_cols = OrderedDict([
        (col, [])
        for col in df_grouped.columns
    ])
    for (start, end, seq, indices) in merged_intervals:
        rows = df_grouped.iloc[list(indices)]
        result_cols["num_sources"].append(rows["num_sources"].sum())
        result_cols["sources"].append("; ".join(rows["sources"].values))
        result_cols["accessible_subsequence_start"].append(start)
        result_cols["accessible_subsequence_end"].append(end)
        result_cols["accessible_subsequence"].append(seq)
        result_cols["accessible_subsequence_length"].append(len(seq))
        result_cols["accessible_subsequence_is_4mer_or_longer"].append(len(seq) >= 4)
        IgG = rows["IgG"].mean() 
        result_cols["IgG"].append(IgG > 0)
        IgA = rows["IgA"].mean()
        result_cols["IgA"].append(IgA > 0)
        result_cols["IgG_and_IgA"].append(IgG > 0 and IgA > 0)
        result_cols["accessible_subsequence_contains_polymorphism"].append(
            rows["accessible_subsequence_contains_polymorphism"].any())
        result_cols["accessible_subsequence_contains_glycosite"].append(
            rows["accessible_subsequence_contains_glycosite"].any())
        result_cols["near_any_feature"].append(rows["near_any_feature"].any())
        result_cols["in_any_feature"].append(rows["in_any_feature"].all())
    return pd.DataFrame(result_cols)


In [208]:
print("Filtered %d individual B-cell epitope entries into %d accessible regions" % (
    len(df_sars2_S),
    len(df_grouped)))


df_grouped_overlapping = combine_overlapping_epitopes(df_grouped)
df_grouped_overlapping.to_csv("accessible-linear-bcell-epitopes-merged-overlap.csv", index=False)

print("Collapsed overlapping accessible regions into %d/%d epitopes" % (
    len(df_grouped_overlapping),
    len(df_grouped)))


df_no_glycosites = df_grouped_overlapping[~df_grouped_overlapping.accessible_subsequence_contains_glycosite]

print("Glycosite filter: %d/%d " % (
    len(df_no_glycosites), len(df_grouped_overlapping)))

df_no_polymorphisms = df_no_glycosites[~df_no_glycosites.accessible_subsequence_contains_polymorphism]

print("Polymorphism filter: %d/%d" % (
    len(df_no_polymorphisms), len(df_no_glycosites)))

df_near_feature = df_no_polymorphisms[df_no_polymorphisms.near_any_feature]

print("Near RBD (+/- 30aa), FP (+/- 10aa), HR1 (+/- 10aa), HR2 (+/- 10aa): %d/%d" % (
    len(df_near_feature), len(df_no_polymorphisms)))


df_accessible_4mer = df_near_feature[df_near_feature.accessible_subsequence_is_4mer_or_longer]

print("4mer or longer: %d/%d" % (
    len(df_accessible_4mer), len(df_near_feature)))

df_accessible_4mer.to_csv("accessible-linear-bcell-epitopes-grouped-merged-filtered.csv");


Filtered 54 individual B-cell epitope entries into 26 accessible regions
Collapsed overlapping accessible regions into 22/26 epitopes
Glycosite filter: 19/22 
Polymorphism filter: 16/19
Near RBD (+/- 30aa), FP (+/- 10aa), HR1 (+/- 10aa), HR2 (+/- 10aa): 9/16
4mer or longer: 4/9


In [209]:
df_accessible_4mer

,num_sources,sources,accessible_subsequence_start,accessible_subsequence_end,accessible_subsequence,accessible_subsequence_length,accessible_subsequence_is_4mer_or_longer,IgG,IgA,IgG_and_IgA,accessible_subsequence_contains_polymorphism,accessible_subsequence_contains_glycosite,near_any_feature,in_any_feature
9,3,Wang 2020 S456-460; Dahlke 2020 S449-463; Char...,457,473,RKSNLKPFERDISTEIY,17,True,True,True,True,False,False,True,True
12,4,Charite 2020 S569-583; Charite 2020 S571-585; ...,580,583,QTLE,4,True,True,True,True,False,False,True,False
17,6,Poh 2020 S809-826; Wang 2020 S806-820; Charite...,809,812,PSKP,4,True,True,True,True,False,False,True,False
21,2,Dahlke 2020 S1131-1145; Charite 2020 S1141-1155,1142,1146,QPELD,5,True,True,True,True,False,False,True,False
